---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [19]:
num = 5     # number of bonds
maturities   = [2, 4, 6, 8, 10]                 # enter a list of bond maturities
coupon_rates = [0.04, 0.04, 0.04, 0.04, 0.04]   # enter a list of coupon rates (annual in decimal notation)
yields       = [0.02, 0.03, 0.04, 0.05, 0.06]   # enter a list of yield (annual in decimal notation)

if all(len(lst)==num for lst in [maturities, coupon_rates, yields])==False:
    print('Lists need to be of the same length!')

# FUNCTIONS

In [20]:
import numpy as np
from scipy.optimize import minimize

def price(**kwargs):
    n = int(2 * kwargs['maturity'])
    c = 100 * kwargs['coupon'] / 2
    if 'yld' in kwargs.keys():
        yld = kwargs['yld'] / 2
        return c * np.sum((1 + yld) ** np.arange(-1, -n - 1, -1)) + 100 / (1 + yld) ** n
    else:
        spots = np.array(kwargs['spots']) / 2
        return c * np.sum((1 + spots[:n]) ** np.arange(-1, -n - 1, -1)) + 100 / (1 + spots[n - 1]) ** n

def error(bond, spots):
    p = price(**bond)
    m = bond['maturity']
    n = int(2 * m)
    phat = price(maturity=m, coupon=bond['coupon'], spots=spots[:n])
    return np.log(phat / p)

def forward_rates(spots):
    future_factors = (1+spots/2)**np.arange(1,len(spots)+1)
    change_logs = np.diff(np.log(future_factors))
    f = 2 * (np.exp(change_logs)-1)
    return np.concatenate(([spots[0]], f))

def objective(bonds, spots):
    sse = np.sum([error(bond, spots)**2 for bond in bonds])
    forwards = forward_rates(spots)
    diffs = np.sum(np.diff(np.log(1+forwards/2))**2)
    return sse + 0.5*diffs

def spot_rates(bonds):
    maturities = [bond['maturity'] for bond in bonds]
    n = int(2*np.max(maturities))
    result = minimize(lambda x: objective(bonds, x), [0.05]*n)
    return result.x if result.success==True else np.nan

# CALCULATIONS

In [21]:
bonds = [dict(
        maturity= maturities[i],
        coupon  = coupon_rates[i],
        yld     = yields[i]
    )
    for i in range(num)
]

maxmaturity = np.max([bond['maturity'] for bond in bonds])
spots    = spot_rates(bonds)
forwards = forward_rates(spots)


# FIGURE

In [22]:
import plotly.graph_objects as go

# yields of input bonds
trace1 = go.Scatter(
    x=[bond['maturity'] for bond in bonds],
    y=[bond['yld'] for bond in bonds],
    name='yield',
    mode='markers',
    marker=dict(size=10, color="red")
)

# spot rates
trace2 = go.Scatter(
    x=np.arange(0.5, maxmaturity + 0.5, 0.5),
    y=spots,
    name='spot rate',
    mode='markers+lines',
    marker=dict(color="green"),
    line=dict(color="green")
)

# forward rates
trace3 = go.Scatter(
    x=np.arange(0.5, maxmaturity + 0.5, 0.5),
    y=forwards,
    name='forward rate',
    mode='markers+lines',
    marker=dict(color="blue"),
    line=dict(color="blue")
)
fig = go.Figure()
fig.add_trace(trace3)
fig.add_trace(trace2)
fig.add_trace(trace1)

# Formatting
ymax = np.max([bond['yld'] for bond in bonds])
ymin = np.min([bond['yld'] for bond in bonds])
smax = np.max(spots)
smin = np.min(spots)
fmax = np.max(forwards)
fmin = np.min(forwards)
amax = np.max([ymax,smax,fmax])
amin = np.min([ymin,smin,fmin])
amid = (amax+amin)/2
amax = np.max([amax+0.01, amid+0.02])
amin = np.min([amin-0.01,amid-0.02])
fig.update_layout(
    hovermode="x unified",
    xaxis_rangemode="tozero",
    yaxis_tickformat=".2%",
    yaxis=dict(range=[amin,amax]),
    xaxis_title="Maturity (years)",
    yaxis_title="Rate",
    template="plotly_white",
    legend=dict(
        yanchor="top", 
        y=0.99, 
        xanchor="left", 
        x=0.01)   
)
fig.show()